# Loading some packages

In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
from numba import njit
from pathlib import Path
import gc

INSTRUMENT = 'usdchf'

# Loading data

In [2]:
data_path = Path(r"C:/\Users/\vchar/\OneDrive/\Desktop/\ML Projects/\portfolio/\PublicTradingBot/\experiments/\merged_mt5_data.csv")

main_df = pd.read_csv(data_path)
main_df['datetime'] = pd.to_datetime(main_df['datetime'])
selected_cols = ['datetime']
selected_cols.extend(
    [col for col in main_df.columns if INSTRUMENT in col]
)
main_df = main_df[selected_cols]
main_df.head()

,datetime,usdchf_open,usdchf_high,usdchf_low,usdchf_close,usdchf_tv,usdchf_spread,usdchf_rv
0,2015-11-17 07:00:00+00:00,1.01101,1.01130,1.01090,1.01128,703.0,10.0,6.699500e+08
1,2015-11-17 08:00:00+00:00,1.01126,1.01164,1.01110,1.01123,1270.0,9.0,9.757940e+08
2,2015-11-17 09:00:00+00:00,1.01127,1.01253,1.01104,1.01151,3989.0,9.0,3.140130e+09
3,2015-11-17 10:00:00+00:00,1.01152,1.01316,1.01141,1.01286,4926.0,9.0,3.578041e+09
4,2015-11-17 11:00:00+00:00,1.01285,1.01351,1.01076,1.01111,4746.0,9.0,3.484450e+09


In [3]:
train_df = main_df[main_df['datetime'].dt.year.isin(list(np.arange(2015, 2022)))]
valid_df = main_df[main_df['datetime'].dt.year.isin(list(np.arange(2022, 2024)))]
test_df = main_df[main_df['datetime'].dt.year.isin(list(np.arange(2024, 2026)))]

print(train_df.shape, valid_df.shape, test_df.shape)

(53681, 8) (17520, 8) (9989, 8)


# Creating supplementary functions

In [4]:
@njit(cache=True)
def get_lag(prices, lag=1):
    
    n = len(prices)
    result = np.full((n, 1), np.nan, dtype=np.float64)  # Initialize with -999

    for i in range(lag, n):
        result[i] = prices[i - lag]

    return result

@njit(cache=True)
def moving_min(arr, window):

    result = np.full((len(arr), 1), np.nan, dtype=arr.dtype)
    
    for i in range(len(arr)):
        if i + 1 >= window:
            result[i] = np.min(arr[i + 1 - window:i + 1])

    return result

@njit(cache=True)
def moving_max(arr, window):

    result = np.full((len(arr), 1), np.nan, dtype=arr.dtype)
    
    for i in range(len(arr)):
        if i + 1 >= window:
            result[i] = np.max(arr[i + 1 - window:i + 1])

    return result

# Testing

In [5]:
def strategy1(close, n_lags, ind_window):

    entry = np.where(((close < get_lag(close, lag=n_lags))), -1, 0)
    entry = np.where(((close > get_lag(close, lag=n_lags))), 1, entry)

    # buy_entries = get_lag((entry == 1), lag=1)
    buy_exits = (close == moving_min(close, window=ind_window))

    # sell_entries = get_lag((entry == -1), lag=1)
    sell_exits = (close == moving_max(close, window=ind_window))

    buy_exits = buy_exits.reshape(-1, )
    sell_exits = sell_exits.reshape(-1, )

    return entry, buy_exits, sell_exits

s1_indicator = vbt.IndicatorFactory(
    class_name='Strategy1',
    short_name='s1',
    input_names=['close'],
    param_names=['n_lags', 'ind_window'],
    output_names=['value', 'buy_exit', 'sell_exit']
).from_apply_func(strategy1, n_lags=1, ind_window=7)

In [ ]:
# lags_values = list(np.arange(1, 150, 1))
# bars_values = list(np.arange(1, 150, 1))
lags_values = list(np.arange(501, 1001, 1))
# bars_values = list(np.arange(150, 300, 1))
window_values = list(np.arange(501, 1001, 1))


s1_entries = s1_indicator.run(
    train_df[f'{INSTRUMENT}_close'],
    n_lags=lags_values,
    ind_window=window_values,
    param_product=True
)
buy_entries = (s1_entries.value == 1).vbt.fshift(1)
sell_entries = (s1_entries.value == -1).vbt.fshift(1)
buy_exits = s1_entries.buy_exit
sell_exits = s1_entries.sell_exit

s1_pf = vbt.Portfolio.from_signals(
    train_df[f'{INSTRUMENT}_open'], 
    entries=buy_entries, 
    exits=buy_exits,
    init_cash=100000,
    fees=0.015,
    slippage=0.00005,
    size=0.1,
    short_entries=sell_entries,
    short_exits=sell_exits
)

# s1_pf.stats()
returns = s1_pf.total_return()

print(returns.max(), returns.idxmax())

gc.collect()